### Дополнительное задание

<details>
  <summary>Формулировка</summary>
  
  1. **Загрузите датасет оценки качества воздуха в различных регионах**  
     Скачать [air_quality.csv](https://disk.yandex.ru/d/29AVoENYZR4NyA)
  
  2. **Подготовьте данные к обучению** - **1 балл**
     - Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией.
     - В качестве целевой переменной используйте столбец `air_quality` (бинарный столбец: 0 - плохое качество воздуха, 1 - хорошее качество воздуха).
     - Создайте объекты для работы с данными в PyTorch: `Dataset` и `DataLoader` для обучающей, валидационной и тестовой выборок. Выберите оптимальный `batch_size`.
  
  3. **Реализуйте класс нейросетевой модели для решения задачи** - **1 балл**
     - Минимальное количество `Linear` слоев в структуре - 3 штуки: входной слой, скрытый слой, выходной классификационный слой.
     - Подберите оптимальные функции активации: `ReLU`, `Sigmoid`, `Tanh`, `LeakyReLU`.
     - Реализуйте логику прохождения данных по сети в методе `forward`.
     - Создайте объект модели и реализуйте перевод модели на **GPU**.
  
  4. **Напишите код цикла обучения - train-loop и валидации - eval-loop** - **1 балл**
     - В процессе обучения сохраняйте значения функции потерь на тренировочной и валидационной выборках.
  
  5. **Обучите модель и проверьте ее качество** - **1 балл**
     - Выберите оптимизатор, в качестве функции потерь используйте `nn.BCELoss`.
     - Запустите обучение, подберите оптимальные параметры: скорость обучения и количество эпох, ориентируясь на динамику функции потерь на `train/val`.
     - Измерьте качество лучшей модели на тестовой выборке, постройте отчет о классификации с использованием `classification_report`.
  
  6. **Обеспечена воспроизводимость решения** - **1 балл**
     - Зафиксированы `random_state`, ноутбук воспроизводится от начала до конца без ошибок.
</details>

In [18]:
# импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch.utils.data import Dataset, DataLoader

### 1. Подготовка данных к обучению

In [8]:
# Загружаем датасет
data = pd.read_csv("../data/air_quality.csv")
data.head()

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,air_quality
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,1
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,1
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,1
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,1
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,1


In [9]:
# Разделяем признаки (X) и целевую переменную (y)
X = data.drop(columns=["air_quality"])
y = data["air_quality"]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Temperature                    5000 non-null   float64
 1   Humidity                       5000 non-null   float64
 2   PM2.5                          5000 non-null   float64
 3   PM10                           5000 non-null   float64
 4   NO2                            5000 non-null   float64
 5   SO2                            5000 non-null   float64
 6   CO                             5000 non-null   float64
 7   Proximity_to_Industrial_Areas  5000 non-null   float64
 8   Population_Density             5000 non-null   int64  
 9   air_quality                    5000 non-null   int64  
dtypes: float64(8), int64(2)
memory usage: 390.8 KB


In [11]:
data.describe()

,Temperature,Humidity,PM2.5,PM10,NO2,SO2,CO,Proximity_to_Industrial_Areas,Population_Density,air_quality
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,30.029020,70.056120,20.142140,30.218360,26.412100,10.014820,1.500354,8.425400,497.423800,0.700000
std,6.720661,15.863577,24.554546,27.349199,8.895356,6.750303,0.546027,3.610944,152.754084,0.458303
min,13.400000,36.000000,0.000000,-0.200000,7.400000,-6.200000,0.650000,2.500000,188.000000,0.000000
25%,25.100000,58.300000,4.600000,12.300000,20.100000,5.100000,1.030000,5.400000,381.000000,0.000000
50%,29.000000,69.800000,12.000000,21.700000,25.300000,8.000000,1.410000,7.900000,494.000000,1.000000
75%,34.000000,80.300000,26.100000,38.100000,31.900000,13.725000,1.840000,11.100000,600.000000,1.000000
max,58.600000,128.100000,295.000000,315.800000,64.900000,44.900000,3.720000,25.800000,957.000000,1.000000


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.4, stratify=y, random_state=21)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=21)